## Session 1: Pandas Basics

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

print(pd.__version__)
print(np.__version__)

### What is Pandas?

- [Pandas](http://pandas.pydata.org/) is a package that provides high-performance, easy-to-use data structures and data analysis tools for Python.


 <img src="pandas_vs_rest.png"> 

### Loading the dataset into memory

In [ ]:
stocks_filename = 'stocks_data.csv'
stocks_df = pd.read_csv(stocks_filename)

### Inspecting the dataset

In [ ]:
stocks_df

In [ ]:
stocks_df.head(10)

# Note: this returns a new dataframe

In [ ]:
stocks_df.tail(10)

### DataFrame vs NumPy array

In [ ]:
stocks_df.values

In [ ]:
stocks_df.index

In [ ]:
stocks_df.columns

### Selecting columns and rows

In [ ]:
# Selecting a single column
stocks_df['ticker']

# Get the numpy array from Series
# stocks_df['ticker'].values

# Get Series index
# stocks_df['ticker'].index

# stocks_df.ticker

In [ ]:
# Multiple column selection
stocks_df[['ticker', 'adj_close']]

In [ ]:
# Row selection
stocks_df[0]  # Doesn't work with DataFrame but works with Series

# stocks_df.ticker[0]

# stocks_df.ticker[-1]

In [ ]:
# Label based special operator
stocks_df.loc[10:20, ['ticker', 'adj_close']]

In [ ]:
# Position based special operator
stocks_df.iloc[10:20, [0, 3]]

In [ ]:
stocks_df[:10]

### Basic Operations

In [ ]:
# Mathematical operations
stocks_df['high'] - stocks_df['low']

# stocks_df * 2
# np.log(stocks_df.adj_close)

In [ ]:
# Comparisons
stocks_df.volume > 100_000_000

### Filtering

In [ ]:
# Single condition
condition = stocks_df.volume > 100_000_000
stocks_df[condition]

In [ ]:
# Multiple conditions
condition = stocks_df.volume > 100_000_000 and stocks_df.adj_close > 10.   # Doesn't work because you can't overide `and`
# condition = stocks_df.volume > 100_000_000 & stocks_df.adj_close > 10.    # Doesn't work because `&` has precedence
# condition = (stocks_df.volume > 100_000_000) & (stocks_df.adj_close > 10.)   # Works

In [ ]:
stocks_df[condition].ticker.unique()

### Manipulating strings

In [ ]:
stocks_df.head()

In [ ]:
'Economics'[:4]

In [ ]:
# stocks_df.date[:4]
# stocks_df.date.str[:4]

In [ ]:
stocks_df[stocks_df.date.str.startswith('2018')]

### Map

In [ ]:
stocks_df.head()

In [ ]:
company_name_to_industry = pd.Series(data=stocks_df.industry.values, index=stocks_df.name.values).to_dict()
company_name_to_industry

In [ ]:
stocks_df.name.map(company_name_to_industry)

In [ ]:
pd.qcut(stocks_df.volume, 5).cat.categories

In [ ]:
def volume_classifier(volume):
    if volume < 363500.0:
        return 'Very small'
    elif volume < 963000.0:
        return 'Small'
    elif volume < 2001200.0:
        return 'Medium'
    elif volume < 4497400.0:
        return 'Large'
    else:
        return 'Very large'
    
stocks_df.volume.map(volume_classifier)

### Plotting

In [ ]:
stocks_df[stocks_df.ticker == 'AAPL'].adj_close.plot()

In [ ]:
stocks_df[stocks_df.ticker.isin(['AAPL', 'NFLX', 'AMZN', 'MSFT'])].ticker.value_counts().plot(kind='bar')

### Indexing

In [ ]:
df1 = stocks_df.set_index('ticker')
df1

In [ ]:
df1.reset_index()

In [ ]:
df2 = stocks_df.set_index(['industry', 'ticker'])
df2

In [ ]:
stocks_df.set_index(['industry', 'ticker']).loc['COMPUTER MANUFACTURING']

### Fast Lookups using Indices

In [ ]:
%%time
stocks_df[stocks_df.ticker == 'AAPL']

In [ ]:
df1 = stocks_df.set_index('ticker').sort_index()
df2 = df1.sample(frac=1)       # Shuffle the index

In [ ]:
df2

In [ ]:
%%time
df2.loc['AAPL']

In [ ]:
%%time
df1.loc['AAPL']

## Session 2: Data Wrangling

### Concat

`pd.concat` concatenates or "stacks" together objects along an axis.

In [ ]:
df1 = stocks_df[:200]
df2 = stocks_df[200:400]

In [ ]:
pd.concat([df1, df2])

In [ ]:
df1 = stocks_df.iloc[:, 0:3]
df2 = stocks_df.iloc[:, 3:6]

pd.concat([df1, df2], axis=1)

### Merge

`pd.merge` connects rows in DataFrames based on one or more keys.

Three types of merge:
- One-to-one merge
- Many-to-one merge
- Many-to-many merge

In [ ]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [ ]:
# One-to-one merge
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

In [ ]:
df3 = pd.merge(df1, df2)
df3

In [ ]:
df2.columns = ['name', 'hire_date']
pd.merge(df1, df2, left_on='employee', right_on='name')

In [ ]:
# Many-to-one merge
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

In [ ]:
# Many-to-many
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

In [ ]:
# Set of groups is different
df6 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Alex'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'Intern']})

df7 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR', 'Management'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization', 'people']})
display('df6', 'df7')

<img src="pandas_merge.png"> 

In [ ]:
pd.merge(df6, df7, how='outer')

In [ ]:
np.random.seed(1)

set1 = np.random.choice(stocks_df.ticker.unique(), size=50)
set2 = np.random.choice(stocks_df.ticker.unique(), size=50)

df1 = stocks_df.loc[stocks_df.ticker.isin(set1), ['ticker', 'adj_close', 'volume']]
df2 = stocks_df.loc[stocks_df.ticker.isin(set2), ['ticker', 'name', 'industry']].drop_duplicates()

In [ ]:
pd.merge(df1, df2)

In [ ]:
len(set(df1.ticker) & set(df2.ticker))

In [ ]:
pd.merge(df1, df2).ticker.unique()

## Session 3: Data Aggregation and Group Operations

### Split-Apply-Combine

<img src="split_apply_combine.png"> 

### Groupby

In [ ]:
stocks_df.groupby('exchange')

In [ ]:
stocks_df.groupby('exchange').ticker.unique()

In [ ]:
stocks_df[stocks_df.exchange == 'NASDAQ'].ticker.unique()
# stocks_df[stocks_df.exchange == 'NYSE'].ticker.unique()

In [ ]:
# Groupby can be done using a Series that is not in the dataframe
stocks_df.groupby(stocks_df.exchange).ticker.unique()

In [ ]:
stocks_df.groupby(['exchange', 'date']).volume.sum()

In [ ]:
stocks_df.groupby(['exchange', 'date']).volume.agg(['sum', 'max', 'min'])

### Stack and Unstack

In [ ]:
stocks_df['year'] = stocks_df.date.str[0:4]
stocks_df['month'] = stocks_df.date.str[5:7]
monthly_perf = \
stocks_df.groupby(['ticker', 'year', 'month']).adj_close.last() / stocks_df.groupby(['ticker', 'year', 'month']).adj_close.first() - 1
monthly_perf

In [ ]:
monthly_perf.unstack()

In [ ]:
monthly_perf.unstack().swaplevel('ticker', 'year')

In [ ]:
monthly_perf.unstack(level=0)
# monthly_perf.unstack(level=1)
# monthly_perf.unstack('month')

In [ ]:
# Caveat
monthly_perf.unstack().unstack()
# monthly_perf.unstack().unstack().unstack()  # Gives you a Series because you no longer have 2 dimensions

### Apply

In [ ]:
def draw(df, n=10):
    return df.sample(n)
    
stocks_df.groupby('ticker').apply(draw, n=50)

### Topics not covered

- Data loading
- Handling missing values 
- Time Series

## References

- Brandon Rhodes - Pandas From The Ground Up - PyCon 2015: https://www.youtube.com/watch?v=5JnMutdy6Fw
- Natasha Watkins' Pandas Tutorial: https://github.com/QuantEcon/ShenzhenWinterCamp/tree/master/Natasha
- QuantEcon Pandas' lecture
- Pandas Documentation: https://pandas.pydata.org/pandas-docs/stable/index.html
- Python for Data Analysis, Wes McKinney
- [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)